## Introducción

En este proyecto analizaré la base de datos de una tienda online de venta de videojuegos, con actividades en varios países. 
Mi objetivo principal es identificar patrones que determinen cuales son los juegos más exitosos y cuales no, lo cual permitirá plantear nuevos proyectos y diseñar campañas publicitarias eficaces para el próximo año. 

Con mi objetivo en mente, realizaré un análisis exploratorio de datos y prepararé el conjunto para trabajar con el; plantearé algunas hipótesis enfocadas en los números arrojados por las ventas en los diferentes países y las críticas de los usuarios y la junta de entretenimiento; mediante la prueba de esas hipótesis determinaré cuales videojuegos y las plataformas son más exitosas. 

## Inicialización

In [1]:
#importar librerías
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt
from math import factorial
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

## Cargar datos

In [2]:
#leer el conjunto de datos en un dataframe
games = pd.read_csv("dataset/games.csv")

## Preparar los datos

In [3]:
#imprimir información general del dataframe
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [4]:
#imprimir muestra de datos en el dataframe
games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


## Corregir los datos

In [5]:
#reemplazar nombres de columnas a minúsculas
new_col_names = []

for old_name in games.columns:
    name_lowered = old_name.lower()
    new_col_names.append(name_lowered)
    
games.columns = new_col_names


In [6]:
# reemplazar tipos de datos en dos columnas
games['year_of_release'] = games['year_of_release'].astype('Int64')

games['user_score'] = pd.to_numeric(games['user_score'], errors='coerce')

Cambié el tipo de datos en la columna 'year_of_release' a números enteros; y en la columna 'user_score' cambié los datos de 'string' a 'float' para poder resalizar cálculos con esta última columna más adelante.

### Valores ausentes

In [7]:
#verificar los valores ausentes
games.isna().sum()

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         9125
rating             6766
dtype: int64

In [8]:
#rellenar valores ausentes en "name"
games['name'] = games['name'].fillna('unknown')

In [9]:
#rellenar valores ausentes en "year_of_release"
games['year_of_release'] = games['year_of_release'].fillna(0)

In [10]:
#rellenar valores ausentes en "genre"
games['genre'] = games['genre'].fillna('unknown')

In [11]:
#rellenar valores ausentes en "rating"
games['rating'] = games['rating'].fillna('unknown')


In [12]:
#verificar nuevamente valores ausentes
games.isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         9125
rating                0
dtype: int64

#### Manejo de valores ausentes

Tanto para la columna "name", "genre" y "rating", rellené los valores ausentes con un string 'unknown'.

En la columna "year_of_release" rellené los valores ausentes con 0; cero significando "año desconocido".

Finalmente, para las columnas "critic_score" y "user_score", voy a mantener los valores ausentes como están. Algunas de las filas de estas columnas tenían la abreviación "tbd" ("to be determined"), lo que me hace pensar que al momento de extraer la base de datos, aun no se habían calificado. Por otra partes, como los videojuegos pertencen a géneros diferentes, no rellenaré los valores ausentes con la media, ya que está puede variar de un género a otro. 

#### Ventas totales

In [13]:
#calcular las ventas totales en una nueva columna 

games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']


## Análisis de datos

In [14]:
#calcular cuantos juegos fueron lanzados cada año
games_per_year = games.groupby('year_of_release')['name'].count().reset_index()

games_per_year

,year_of_release,name
0,0,269
1,1980,9
2,1981,46
3,1982,36
4,1983,17
5,1984,14
6,1985,14
7,1986,21
8,1987,16
9,1988,15


Algunos años tienen cifras siginificativas de videojuegos lanzados en esos períodos. Estos años van de 2005 a 2011.